In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets, models
from torchvision.models import vgg11
from sklearn.metrics import precision_score, recall_score

In [38]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [39]:
import zipfile
import os

In [5]:
zip_file_path = "/content/drive/MyDrive/Dataset_Project/Dataset 1.zip"
extracted_folder_path = "/content/drive/MyDrive/Dataset_Project/Dataset_1"

os.makedirs(extracted_folder_path, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [6]:
zip_file_path = "/content/drive/MyDrive/Dataset_Project/Dataset 2.zip"
extracted_folder_path = "/content/drive/MyDrive/Dataset_Project/Dataset_2"

os.makedirs(extracted_folder_path, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [7]:
zip_file_path = "/content/drive/MyDrive/Dataset_Project/Dataset 3.zip"
extracted_folder_path = "/content/drive/MyDrive/Dataset_Project/Dataset_3"

os.makedirs(extracted_folder_path, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [52]:
data1_path = "/content/drive/MyDrive/Dataset_Project/Dataset_1/Dataset 1/Colorectal Cancer "
data2_path = "/content/drive/MyDrive/Dataset_Project/Dataset_2/Dataset 2/Prostate Cancer"
data3_path = "/content/drive/MyDrive/Dataset_Project/Dataset_3/Dataset 3/Animal Faces"

In [53]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

dataset1 = datasets.ImageFolder(data1_path, transform=transform)

train_size = int(0.8 * len(dataset1))
val_size = int(0.1 * len(dataset1))
test_size = len(dataset1) - train_size - val_size
train_set, val_set, test_set = random_split(dataset1, [train_size, val_size, test_size])

In [57]:
model = vgg11(weights='DEFAULT')

for param in model.features[:-5].parameters():
    param.requires_grad = False

num_classes = 3
model.classifier[6] = nn.Linear(4096, num_classes)

In [58]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [59]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32)
test_loader = DataLoader(test_set, batch_size=32)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

best_val_loss = float("inf")
patience = 5
no_improvement_count = 0

num_epochs = 50
for epoch in range(num_epochs):
    print('start epoch ', epoch)
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {running_loss / len(train_loader)}")

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

        val_loss /= len(val_loader)

    print(f'Validation Loss: {val_loss}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= patience:
        print(f'Early stopping after {epoch + 1} epochs with no improvement.')
        break

print('Training finished.')
print('Testing begin.')
model.eval()
correct = 0
total = 0
test_loss = 0.0

all_labels = []
all_predicted = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)

        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
avg_test_loss = test_loss / len(test_loader)
precision = precision_score(all_labels, all_predicted, average='weighted')
recall = recall_score(all_labels, all_predicted, average='weighted')



print(f'Test Accuracy: {accuracy}%')
print(f'Test Loss: {avg_test_loss}')
print('Test Precision', precision)
print('Test recall', recall)

start epoch  0
Epoch 1/50 - Loss: 0.34232230550299086
Validation Loss: 0.19745774410272898
start epoch  1
Epoch 2/50 - Loss: 0.16668879775330425
Validation Loss: 0.12025732841146619
start epoch  2
Epoch 3/50 - Loss: 0.10458054318713646
Validation Loss: 0.12941341504062476
start epoch  3
Epoch 4/50 - Loss: 0.0739613344706595
Validation Loss: 0.0943769721412345
start epoch  4
Epoch 5/50 - Loss: 0.053862351470937334
Validation Loss: 0.10565892705007603
start epoch  5
Epoch 6/50 - Loss: 0.05065047029250612
Validation Loss: 0.1255147883570508
start epoch  6
Epoch 7/50 - Loss: 0.037614879292280724
Validation Loss: 0.07405579569259364
start epoch  7
Epoch 8/50 - Loss: 0.02032962310302537
Validation Loss: 0.0815734692457083
start epoch  8
Epoch 9/50 - Loss: 0.011097274348770347
Validation Loss: 0.06807732745686448
start epoch  9
Epoch 10/50 - Loss: 0.005271138357131425
Validation Loss: 0.07838925934323158
start epoch  10
Epoch 11/50 - Loss: 0.00662055506594091
Validation Loss: 0.09270400946077

TASK 2 Model 1 dataset 2:


In [60]:
model1 = nn.Sequential(*list(model.features.children())[:-5])
model1.to(device)
model1.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (12): ReLU(inplace=True)
  (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (14): ReLU(inplace=True)
  (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [72]:
model12 = model1

In [71]:
dataset2 = datasets.ImageFolder(data2_path, transform=transform)

train_size2 = int(0.8 * len(dataset2))
val_size2 = int(0.1 * len(dataset2))
test_size2 = len(dataset2) - train_size2 - val_size2
train_set2, val_set2, test_set2 = random_split(dataset2, [train_size2, val_size2, test_size2])

In [77]:
train_loader = DataLoader(train_set2, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set2, batch_size=32)
test_loader = DataLoader(test_set2, batch_size=32)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model12.parameters(), lr=0.001, momentum=0.9)

best_val_loss = float("inf")
patience = 5
no_improvement_count = 0

num_epochs = 50
for epoch in range(num_epochs):
    print('start epoch ', epoch)
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {running_loss / len(train_loader)}")

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

        val_loss /= len(val_loader)

    print(f'Validation Loss: {val_loss}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= patience:
        print(f'Early stopping after {epoch + 1} epochs with no improvement.')
        break

print('Training finished.')
print('Testing begin.')
model.eval()
correct = 0
total = 0
test_loss = 0.0

all_labels = []
all_predicted = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)

        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
avg_test_loss = test_loss / len(test_loader)
precision = precision_score(all_labels, all_predicted, average='weighted')
recall = recall_score(all_labels, all_predicted, average='weighted')



print(f'Test Accuracy: {accuracy}%')
print(f'Test Loss: {avg_test_loss}')
print('Test Precision', precision)
print('Test recall', recall)

start epoch  0
Epoch 1/50 - Loss: 2.6360572735468546
Validation Loss: 2.5516080668098047
start epoch  1
Epoch 2/50 - Loss: 2.6124315985043842
Validation Loss: 2.5516080668098047
start epoch  2
Epoch 3/50 - Loss: 2.639640770753225
Validation Loss: 2.5516080668098047
start epoch  3
Epoch 4/50 - Loss: 2.626651151974996
Validation Loss: 2.5516080668098047
start epoch  4
Epoch 5/50 - Loss: 2.5963445965449017
Validation Loss: 2.5516080668098047
start epoch  5
Epoch 6/50 - Loss: 2.59914825518926
Validation Loss: 2.5516080668098047
Early stopping after 6 epochs with no improvement.
Training finished.
Testing begin.
Test Accuracy: 41.333333333333336%
Test Loss: 2.5569603631370947
Test Precision 0.48214632816849134
Test recall 0.41333333333333333


Model 1 Dataset 3

In [78]:
model13 = model1

In [79]:
dataset3 = datasets.ImageFolder(data3_path, transform=transform)

train_size3 = int(0.8 * len(dataset3))
val_size3 = int(0.1 * len(dataset3))
test_size3 = len(dataset3) - train_size3 - val_size3
train_set3, val_set3, test_set3 = random_split(dataset3, [train_size3, val_size3, test_size3])

In [80]:
train_loader = DataLoader(train_set3, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set3, batch_size=32)
test_loader = DataLoader(test_set3, batch_size=32)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model13.parameters(), lr=0.001, momentum=0.9)

best_val_loss = float("inf")
patience = 5
no_improvement_count = 0

num_epochs = 50
for epoch in range(num_epochs):
    print('start epoch ', epoch)
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {running_loss / len(train_loader)}")

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

        val_loss /= len(val_loader)

    print(f'Validation Loss: {val_loss}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= patience:
        print(f'Early stopping after {epoch + 1} epochs with no improvement.')
        break

print('Training finished.')
print('Testing begin.')
model.eval()
correct = 0
total = 0
test_loss = 0.0

all_labels = []
all_predicted = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)

        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
avg_test_loss = test_loss / len(test_loader)
precision = precision_score(all_labels, all_predicted, average='weighted')
recall = recall_score(all_labels, all_predicted, average='weighted')



print(f'Test Accuracy: {accuracy}%')
print(f'Test Loss: {avg_test_loss}')
print('Test Precision', precision)
print('Test recall', recall)

start epoch  0
Epoch 1/50 - Loss: 3.3903044255574546
Validation Loss: 2.967450236019335
start epoch  1
Epoch 2/50 - Loss: 3.43419419447581
Validation Loss: 2.967450236019335
start epoch  2
Epoch 3/50 - Loss: 3.429710636138916
Validation Loss: 2.967450236019335
start epoch  3
Epoch 4/50 - Loss: 3.4225958132743837
Validation Loss: 2.967450236019335
start epoch  4
Epoch 5/50 - Loss: 3.4084727581342062
Validation Loss: 2.967450236019335
start epoch  5
Epoch 6/50 - Loss: 3.40571515083313
Validation Loss: 2.967450236019335
Early stopping after 6 epochs with no improvement.
Training finished.
Testing begin.
Test Accuracy: 37.5%
Test Loss: 3.041347051921644
Test Precision 0.4093263596781973
Test recall 0.375
